In [1]:
# using tiny shakespeare dataset for demo
!curl -O https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1089k  100 1089k    0     0  2544k      0 --:--:-- --:--:-- --:--:-- 2556k


In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [3]:
len(text)

1115394

In [4]:
print(text[:10_000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# uniques chars in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# map chars to ints; character level encoding to keep things simple --> expect long sequences as a result
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # int out
decode = lambda l: "".join([itos[i] for i in l])  # string out
print(encode("H3LLO fam?"))
print(decode(encode("H3LLO fam?")))

[20, 9, 24, 24, 27, 1, 44, 39, 51, 12]
H3LLO fam?


In [7]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype, data[:10_000])

torch.Size([1115394]) torch.int64 tensor([18, 47, 56,  ..., 43, 10,  1])


In [8]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x, y = train_data[:block_size], train_data[1 : block_size + 1]
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [11]:
torch.manual_seed(1337)
batch_size = 4  # independent sequences will we process in parallel
block_size = 8  # max context length for predictions


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(
        len(data) - block_size, (batch_size,)
    )  # batch_size num of random offsets
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")
print(f"inputs: {xb.shape}, {xb}")
print(f"inputs: {yb.shape}, {yb}")
print("========" * 10)
for b in range(batch_size):  # batch dim
    for t in range(block_size):  # time dim
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} target is {target.tolist()}")

inputs: torch.Size([4, 8]), tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
inputs: torch.Size([4, 8]), tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
When input is [24] target is 43
When input is [24, 43] target is 58
When input is [24, 43, 58] target is 5
When input is [24, 43, 58, 5] target is 57
When input is [24, 43, 58, 5, 57] target is 1
When input is [24, 43, 58, 5, 57, 1] target is 46
When input is [24, 43, 58, 5, 57, 1, 46] target is 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] target is 39
When input is [44] target is 53
When input is [44, 53] target is 56
When input is [44, 53, 56] target is 1
When input is [44, 53, 56, 1] target is 58
When input is [44, 53, 56, 1, 58] target is 46
When input is [44, 53, 56, 1, 58, 46] target is 39


In [13]:
import torch.nn as nn
from torch.nn import functional as F


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # e/token directly reads of the logits for
        # the next token from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        # idx and targets are both (B, T) tensor of ints
        logits = self.token_embedding_table(idx)  # (B(atch), T(ime), C(hannel))

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get preds
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becoming (B, C)
            # apply softmax to get probs
            probs = F.softmax(logits, dim=1)  # (B, C)
            # sample from the fist
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled idx to running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)


m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)  # should be around -log(1/65)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.7288, grad_fn=<NllLossBackward0>)
